In [1]:
!pip install transformers
!pip install nlp

     |████████████████████████████████| 1.8 MB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 738 kB 18.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 1.1 MB/s  eta 0:00:01
     |████████████████████████████████| 303 kB 12.1 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893258 sha256=1b1d5a2137c1edfc8c18470fd1c60443f69c4a6d6bec15027e9c5f5aa10a94c2
  Stored in directory: /root/.cache/pip/wheels/7b/78/f4/27d43a65043e1b75dbddaa421b573eddc67e712be4b1c80677
Successfully built sacremoses
     |████████████████████████████████| 1.7 MB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 20.7 MB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 4.7 MB/s  eta 0:00:01
     |████████████████████████████████| 243 kB 13.5 MB/s eta 0:00:01


In [3]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [4]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset, test_dataset = load_dataset('imdb', split=['train', 'test'])
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Downloading:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

|          | 0/0 [00:00<?, ? examples/s]

|          | 0/0 [00:00<?, ? examples/s]

|          | 0/0 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/76cdbd7249ea3548c928bbf304258dab44d09cd3638d9da8d42480d1d1be3743. Subsequent calls will reuse this data.


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [44]:
train_dataset

Dataset(features: {'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None), 'text': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 25000)

In [6]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

batch_size = 8

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [7]:
trainer.train()

/opt/conda/envs/pytorch/lib/python3.8/site-packages/nlp/utils/py_utils.py:191: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return function(data_struct)
/opt/conda/envs/pytorch/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,0.220000,0.174424,0.933640,0.934944,0.916929,0.953680,244.283500,102.340000


/opt/conda/envs/pytorch/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/envs/pytorch/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1042, training_loss=0.2821401845013111, metrics={'train_runtime': 1039.3349, 'train_samples_per_second': 1.003, 'total_flos': 8408354150400000, 'epoch': 1.0})

In [8]:
trainer.evaluate()

/opt/conda/envs/pytorch/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.17442423105239868,
 'eval_accuracy': 0.93364,
 'eval_f1': 0.934943727696953,
 'eval_precision': 0.9169294669640797,
 'eval_recall': 0.95368,
 'eval_runtime': 241.6786,
 'eval_samples_per_second': 103.443,
 'epoch': 1.0}

In [59]:
pt_batch = tokenizer(
     [
         "I personaly really hated the movie",
         "This movie really sucks",
         "This is the best movie have ever seen"
     ],
     padding=True,
     truncation=True,
     max_length=512,
     return_tensors="pt"
)
pt_batch.to('cuda')
pt_outputs = model(**pt_batch, output_hidden_states=True, return_dict=True)
print(torch.nn.Softmax(dim=1)(pt_outputs['logits']))
pt_outputs['logits'].argmax(axis=1)

tensor([[0.9769, 0.0231],
        [0.9920, 0.0080],
        [0.0042, 0.9958]], device='cuda:0', grad_fn=<SoftmaxBackward>)


tensor([0, 0, 1], device='cuda:0')

In [60]:
trainer.save_model("BERT_finetuned.bin")

In [ ]:
from transformers import AutoModel
loaded_model = BertForSequenceClassification.from_pretrained("BERT_finetuned.bin")

In [77]:
pt_batch = tokenizer(
     [
         "I personaly really hated the movie",
         "This movie really sucks",
         "This is the best movie have ever seen"
     ],
     padding=True,
     truncation=True,
     max_length=512,
     return_tensors="pt"
)
pt_batch.to('cuda')
pt_outputs = loaded_model(**pt_batch, output_hidden_states=True, return_dict=True)
print(torch.nn.Softmax(dim=1)(pt_outputs['logits']))
pt_outputs['logits'].argmax(axis=1)

tensor([[0.9769, 0.0231],
        [0.9920, 0.0080],
        [0.0042, 0.9958]], device='cuda:0', grad_fn=<SoftmaxBackward>)


tensor([0, 0, 1], device='cuda:0')